In [17]:
# import start_instance to start matpower instance
from matpower import start_instance

# start instance
m = start_instance()

m.eval(
    """
    mpopt = mpoption('verbose', 2);
    mpc = loadcase('case14');
    _r1 = runopf(mpc, mpopt);
    """
)

# fech data to python (.eval is used because .pull is not working in acessing field)
r1_mpc = m.eval(
    "struct("
    " 'baseMVA', _r1.baseMVA, 'version', _r1.version, 'bus', _r1.bus, 'gen', _r1.gen,"
    " 'branch', _r1.branch, 'gencost', _r1.gencost);"
)



MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0     13212.573                0.624343        0.275      18.5289            0
  1     8077.4637    0.77403    0.0237554    0.0669394      6.14623     0.221221
  2     8332.0151    0.14646   5.5179e-05    0.0423581      1.61662    0.0140811
  3     8272.2352   0.095342  6.05388e-05     0.058572     0.193778   0.00326095
  4     8132.6028    0.38603  0.000981254   0.00654534    0.0253149   0.00764178
  5     8108.5602     0.1963  0.000743622   0.00106529   0.00469505   0.00132593
  6     8086.8978    0.13748  0.000934896   0.00124858   0.00124516   0.00119625
  7     8083.3486   0.085386  0.000153596   0.00183126  0.000400753  0.000196232
  8     8

In [18]:
#normal runpf (example without modification)
mpc = m.eval('case14', verbose=False)
res = m.runpf(mpc)
result = m.runpf(r1_mpc)


MATPOWER Version 8.0, 17-May-2024
Power Flow -- AC-polar-power formulation

Newton's method converged in 2 iterations.
PF successful

Converged in 0.35 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses             14     Total Gen Capacity     772.4         -52.0 to 148.0
Generators         5     On-line Capacity       772.4         -52.0 to 148.0
Committed Gens     5     Generation (actual)    272.4              82.4
Loads             11     Load                   259.0              73.5
  Fixed           11       Fixed                259.0              73.5
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             1     Shunt (inj)             -0.0              21.2
Branches          20     Losses (I^2 * Z)        13.39             54.54
Transformers    

In [19]:
# modify variable GEN_STATUS in gen matrix
# GEN_STATUS is the 8th column in gen matrix, so we can use m.idx
[GEN_BUS, PG, QG, QMAX, QMIN, VG, MBASE, GEN_STATUS, PMAX, PMIN, MU_PMAX, 
 MU_PMIN, MU_QMAX, MU_QMIN, PC1, PC2, QC1MIN, QC1MAX, QC2MIN, QC2MAX, 
 RAMP_AGC, RAMP_10, RAMP_30, RAMP_Q, APF] = m.idx_gen(nout='max_nout')

In [20]:
# disable one generator 
gen_index = 5 # index of generator to be changed
gen_index_ = int(gen_index - 1) # -1 due to python indexing start from 0
GEN_STATUS_ = int(GEN_STATUS - 1) # -1 due to python indexing start from 0
r1_mpc['gen'][gen_index_,GEN_STATUS_] = 0 

In [21]:
#Check the status of generators
print(r1_mpc['gen'][:, GEN_STATUS_])

[1. 1. 1. 1. 0.]


In [22]:
#re-run power flow with modified generator
# push back value to octave client
m.push('mpc', r1_mpc) # push r1_mpc in python to mpc in octave

# test if we can retrive pushed value
mpc = m.pull('mpc')

# test if our pushed variable can be used
m.eval("_r1 = runopf(mpc, mpopt);")


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0     11187.573                0.624343        0.275      17.0263            0
  1     8020.5976    0.71585    0.0279887    0.0176324       5.1976     0.149473
  2     8314.5657    0.17793  0.000138156    0.0578481      1.51169    0.0163129
  3     8251.3099   0.091621   3.7161e-05    0.0646151     0.179929   0.00345385
  4     8129.2538    0.33863  0.000682791    0.0105826    0.0296119   0.00668752
  5     8108.4779    0.17954  0.000477975   0.00137357   0.00357705   0.00114599
  6      8089.949    0.12654  0.000561391  0.000387569  0.000877312   0.00102322
  7     8087.1206   0.083587  0.000237611   0.00204861  0.000356144  0.000156351
  8     8

In [23]:
#if you want to disable more than one generator, you can use a loop

# Disable generators from index 2 to 4, and enable all others except those
for i in range(r1_mpc['gen'].shape[0]):
    if i in range(3, 5):  # Python index 1,2,3 corresponds to gen 2,3,4
        r1_mpc['gen'][i, GEN_STATUS_] = 0
    else:
        r1_mpc['gen'][i, GEN_STATUS_] = 1

In [24]:
#Check the status of generators
print(r1_mpc['gen'][:, GEN_STATUS_])

[1. 1. 1. 0. 0.]


In [25]:
#re-run power flow with modified generator
# push back value to octave client
m.push('mpc', r1_mpc) # push r1_mpc in python to mpc in octave

# test if we can retrive pushed value
mpc = m.pull('mpc')

# test if our pushed variable can be used
m.eval("_r1 = runopf(mpc, mpopt);")


MATPOWER Version 8.0, 17-May-2024
Optimal Power Flow -- AC-polar-power formulation
MATPOWER Interior Point Solver -- MIPS, Version 1.5.1, 10-May-2024
 (using built-in linear solver)
 it    objective   step size   feascond     gradcond     compcond     costcond
----  ------------ --------- ------------ ------------ ------------ ------------
  0     9162.5731                0.624343        0.275      15.5237            0
  1     7967.0751    0.76102    0.0354973    0.0855965      4.18528    0.0623871
  2     8300.8678    0.26719  0.000817292     0.117032      1.47496     0.018578
  3     8228.3674   0.083314  3.12096e-05    0.0634949     0.203143   0.00396158
  4     8116.8406    0.23224  0.000241915    0.0236252    0.0274954   0.00611831
  5     8099.6728    0.16079  0.000225031   0.00238822   0.00336756  0.000947612
  6      8093.829   0.052677  8.18116e-05  9.74085e-05  0.000473312   0.00032287
  7     8092.4219   0.082811  5.09961e-05   0.00256581   0.00021458  7.77661e-05
  8     8